# Determine Costas Array Frequencies, Order 8, For Yaesu Radios
## By Terry Bondy, VA3TYB

In [1]:
%plot --format svg

In [2]:
source("../common/preamble.m")

Last updated: Tuesday 25 February 2020

In [3]:
source("../common/radio.m")

radioFreqAudioBwLow =  200
radioFreqAudioBwHigh =  2600
radioFreqAudioBw =  2400
radioFreqAudioMidBand =  721.11


Want to spread the frequencies equidistant across the audio bandpass for the radio, but have it centered one the geometric center of the audio bandpass for the radio. That means:

 1. $N = 8$,
 1. $costas8Freq_{8} = costas8Freq_{1} + costas8FreqDelta * (N - 1)$
 1. $\frac{costas8Freq_{1}}{radioFreqAudioBwLow} = \frac{radioFreqAudioBwHigh}{costas8Freq_{8}}$
 1. $costas8Freq_{8} < radioFreqAudioBwHigh$
 1. $radioFreqAudioBwLow < costas8Freq_{1}$
 
Using 1, 2, 4, and 5 an upper limit can be set on $costas8FreqDelta$.
$costas8FreqDelta < \frac{radioFreqAudioBwHigh - radioFreqAudioBwLow}{7}$
or 

In [4]:
(radioFreqAudioBwHigh - radioFreqAudioBwLow)/7

ans =  342.86


So let set

In [5]:
global costas8FreqDelta = 300

Substituting into 3. and re-arranging

 * $costas8Freq_{1} \cdot costas8Freq_{8} = radioFreqAudioBwLow \cdot radioFreqAudioBwHigh$
 * $costas8Freq_{1} \cdot (costas8Freq_{1} + 300 \cdot 7) = 200 \cdot 2600$
 * $costas8Freq_{1}^{2} + costas8Freq_{1} \cdot 2100 = 520000$
 
Solving for the positive root provides

In [6]:
roots([1, 2100, -520000])

ans =

  -2323.77
    223.77



Checking

In [7]:
(2323.77-223.77)/7

ans =  300


In [8]:
223.77/radioFreqAudioBwLow

ans =  1.1189


In [9]:
radioFreqAudioBwHigh/2323.77

ans =  1.1189


Setting to a reasonable nearby integer

In [10]:
global costas8FreqsCv = [225:costas8FreqDelta:2325](:)
# global costas8FreqsCv = [300:costas8FreqDelta:2400](:)


Need a frequency ordering. From _Modified Costas Signal, Levanon & Mozeson_ here is one:

In [11]:
global costas8FreqIdxCv = [ 1 8 3 6 2 7 5 4 ](:);

In [12]:
function sigValuesCv = costasBlock (...
   sampleRate, ...
   numPulses = 1, ...
   freqOffset = 0, ...
   FreqIdx, ...
   Freqs ...
)
    sigValuesCv = [ 0 ];

    if (size(FreqIdx) != size(Freqs))
      error("Expecting size(FreqIdx) == size(Freqs)")
    endif
    # Make a CV
    Freqs = Freqs(:);
    FreqIdx = FreqIdx(:);

    nFreqs = rows(Freqs);
    deltaFreq = Freqs(2) - Freqs(1);

    # Rejig the frequency array and offset so have continuous phase/value for this algorithm
    # regardless of the freqs values.
    deltaOffset = Freqs(1) - deltaFreq;
    Freqs = Freqs .- deltaOffset;
    freqOffset += deltaOffset;

    TimeCv = [0:1/sampleRate:numPulses/deltaFreq](:);
    nTimeCv = rows(TimeCv)-1;
    TimeLongCv = [0:1/sampleRate:numPulses*nFreqs/deltaFreq](:);
    OffsetPhaseCv = TimeLongCv * freqOffset;
    for idx = 1:1:nFreqs
        timeLongOffset = (idx-1)*nTimeCv+1;
        PhaseCv = TimeCv*Freqs(FreqIdx(idx)) .+ OffsetPhaseCv(timeLongOffset:timeLongOffset+nTimeCv);
        SigLoopCV = i * exp(-i*2*pi*PhaseCv);
        sigValuesCv = vertcat(sigValuesCv, SigLoopCV(2:end));
    endfor
endfunction

In [13]:
function sigValuesCv = costas8Block (...
   sampleRate, ...
   numPulses = 1, ...
   freqOffset = 0 ...
)
  global costas8FreqsCv;
  global costas8FreqIdxCv;
  sigValuesCv = costasBlock(sampleRate, numPulses, freqOffset, costas8FreqIdxCv, costas8FreqsCv);
endfunction

In [14]:
BWchannel = 100000
pulseSamples = pow2(ceil(log2(BWchannel * 2/costas8FreqDelta)))
sampleRate = pulseSamples * costas8FreqDelta

BWchannel =  100000
pulseSamples =  1024
sampleRate =  307200


In [15]:
who

Variables in the current scope:

BWchannel              costas8FreqsCv         radioFreqAudioBwLow
ans                    pulseSamples           radioFreqAudioMidBand
costas8FreqDelta       radioFreqAudioBw       sampleRate
costas8FreqIdxCv       radioFreqAudioBwHigh



In [16]:
values = costas8Block(sampleRate, 2);
size(values)
rows(values)/1024

ans =

   16385       1

ans =  16.001


In [ ]:
plot(1:1:rows(values),values)
% grid on
% grid minor
% axis([-1 2], "tic")

In [ ]:
plot(0:300/sampleRate:300*(rows(values)-1)/sampleRate,values)
grid on
grid minor
axis([2-0.1 2+0.1], "tic")

In [ ]:
function c = normxcorr2 (a, b)
  if (nargin != 2)
    print_usage ();
  endif

  ## If this happens, it is probably a mistake
  if (ndims (a) > ndims (b) || any (postpad (size (a), ndims (b)) > size (b)))
    warning ("normxcorr2: TEMPLATE larger than IMG. Arguments may be swapped.");
  endif

  a = double (a) - mean (a(:));
%  b = double (b) - mean (b(:));
  b = double (b);

  a1 = ones (size (a));
  ar = reshape (a(end:-1:1), size (a));

  c = convn (b, conj (ar), "valid");
  b = convn (b.^2, a1, "valid") .- convn (b, a1, "valid").^2 ./ (prod (size (a)));

  ## remove small machine precision errors after substraction
  b(b < 0) = 0;

  a = sumsq (a(:));
  c = reshape (c ./ sqrt (b * a), size (c));

  c(isinf (c) | isnan (c)) = 0;
endfunction


In [ ]:
TimeCv = [0:1/sampleRate:(rows(values)-1)/sampleRate](:);
plot(TimeCv, values)

In [ ]:
TimeSubpulseCv = [0:1/sampleRate:(2*pulseSamples-1)/sampleRate](:);
rows(TimeSubpulseCv)
plot(TimeSubpulseCv, values(1:2*pulseSamples))

In [ ]:
# Need raised cosine for sampling subpulse
RaisedCosineCv = (1 - cos(2*pi*TimeSubpulseCv*costas8FreqDelta*pulseSamples/(rows(TimeSubpulseCv)-1)))/2;
size(RaisedCosineCv)

In [ ]:
plot(TimeSubpulseCv, RaisedCosineCv)

In [ ]:
function Resp = freqResp (values, RaisedCosineCv, pulseSamples)
    # Each column represents freq response at a given moment in time
    Resp = [];
    fracSubPulse = 4; # Must be a power of 2
    indexIncr = pulseSamples/fracSubPulse;
    freqOvr = 2; # Must be a power of 2
    for idx = 1:indexIncr:rows(values)-(rows(RaisedCosineCv)-1)
        SigLoopCV = [values(idx:idx+rows(RaisedCosineCv)-1) .* RaisedCosineCv](1:end-1);
        FftLoopCV = fftshift(fft(SigLoopCV, rows(SigLoopCV)*freqOvr+1));
        Resp = horzcat(Resp, FftLoopCV);
    endfor
endfunction

In [ ]:
% Resp = rot90(freqResp(values, RaisedCosineCv, pulseSamples), 1);
% size(Resp)
size(values)
Resp = freqResp([zeros(size(values)); values; zeros(size(values))], RaisedCosineCv, pulseSamples);
Resp = Resp .* conj(Resp);
Resp = rot90(Resp, -1);
size(Resp)

In [ ]:
% imshow(Resp(end*3/7:end*4/7,1:end))
% imagesc(linspace(-sampleRate/2, sampleRate/2, columns(Resp)), 0:1:rows(Resp), Resp)
imagesc(Resp(1:end,end*6/13:end*7/13))
% axis([-1000 1000])

In [ ]:
Esort = sort(Resp(:));
size(Esort)

In [ ]:
Ecumm = zeros(rows(Esort), 1);
size(Ecumm)
size(Esort)
Ecumm(1) = Esort(1);
for i = 2:rows(Esort)
  Ecumm(i) = Ecumm(i-1) .+ Esort(i);
endfor
size(Ecumm)

In [ ]:
blart = log10(Ecumm/sum(Esort));
blart(isinf(blart)) = -30;
plot(1:1:rows(Ecumm), blart)

In [ ]:
find(blart >= -15)(1)
Esort(ans)
Esort(end)/ans
blart(1)
blart(end)

In [ ]:
Esort = sort(Resp);
size(Esort)

In [ ]:
% FreqLoopCV = linspace(-sampleRate/2, sampleRate/2, columns(Esort));
% plot(linspace(-sampleRate/2, sampleRate/2, columns(Esort)), Esort(end,1:end), Resp(1,1:end))
plot(linspace(-sampleRate/2, sampleRate/2, columns(Esort)), Resp(93,1:end))
grid on
grid minor
axis([-3000 7000 0 100], "tic")

In [ ]:
find(Resp(1:end,1:end) == max(Resp));

In [ ]:
rows(ans)


In [ ]:
A = Resp;
A(A >= 8.7391e-11) = 1;

In [ ]:

imagesc(A)